In [64]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [65]:
# Đọc dữ liệu (giả sử có file review.csv với các cột user_id, product_id, rating)
review_data = pd.read_csv(r"C:\Users\anhn2\Documents\DJANGO\DA\TIKI\comments_data.csv")

In [66]:
# Chuyển dữ liệu thành ma trận người dùng - sản phẩm (user-item matrix).
n_users = review_data['user_id'].nunique()
n_items = review_data['product_id'].nunique()

In [67]:
# Chuyển đổi user_id và product_id thành chỉ số (index)
user_mapping = {user_id: index for index, user_id in enumerate(review_data['user_id'].unique())}
item_mapping = {product_id: index for index, product_id in enumerate(review_data['product_id'].unique())}

In [68]:
# Ma trận đánh giá
ratings_matrix = np.zeros((n_users, n_items))
for row in review_data.itertuples():
    ratings_matrix[user_mapping[row.user_id], item_mapping[row.product_id]] = row.rating

In [69]:
# Khởi tạo các yếu tố ẩn (user and item factors) và bias
n_factors = 20  # Số lượng yếu tố ẩn

# Các tham số SGD cần thử nghiệm
learning_rates = [0.0001, 0.001, 0.005]  # Giảm giá trị learning rate
regularizations = [0.01, 0.05, 0.1]  # Thử giá trị regularization cao hơn
n_epochs_values = [20, 30, 50]  # Cải thiện số epochs

In [70]:
# Hàm chung cho cả có và không có bias
def sgd(matrix, user_factors, item_factors, user_bias=None, item_bias=None, global_bias=None, learning_rate=0.01, regularization=0.1, n_epochs=100, with_bias=False):
    for epoch in range(n_epochs):
        total_loss = 0
        for i in range(n_users):
            for j in range(n_items):
                if matrix[i, j] > 0:  # Chỉ tính với các giá trị không bằng 0 (có rating)
                    if with_bias:
                        # Tính toán dự đoán (bao gồm bias)
                        prediction = global_bias + user_bias[i] + item_bias[j] + np.dot(user_factors[i, :], item_factors[j, :].T)
                    else:
                        # Tính toán dự đoán không có bias
                        prediction = np.dot(user_factors[i, :], item_factors[j, :].T)
                    
                    # Tính lỗi (error)
                    error = matrix[i, j] - prediction
                    total_loss += error**2
                    
                    # Cập nhật các yếu tố ẩn và bias (nếu có)
                    user_factors[i, :] += learning_rate * (error * item_factors[j, :] - regularization * user_factors[i, :])
                    item_factors[j, :] += learning_rate * (error * user_factors[i, :] - regularization * item_factors[j, :])
                    
                    if with_bias:
                        user_bias[i] += learning_rate * (error - regularization * user_bias[i])
                        item_bias[j] += learning_rate * (error - regularization * item_bias[j])
        
        print(f"Epoch {epoch + 1}/{n_epochs} - Loss: {total_loss}")
    
    return user_factors, item_factors, user_bias, item_bias
# Hàm tính RMSE (Root Mean Squared Error) để đánh giá mô hình
def rmse(matrix, predictions):
    non_zero_indices = matrix > 0
    mse = mean_squared_error(matrix[non_zero_indices], predictions[non_zero_indices])
    return np.sqrt(mse)

In [71]:
# Hàm tính RMSE (Root Mean Squared Error) để đánh giá mô hình
def rmse(matrix, predictions):
    non_zero_indices = matrix > 0
    mse = mean_squared_error(matrix[non_zero_indices], predictions[non_zero_indices])
    return np.sqrt(mse)

In [72]:
# Hàm chạy thí nghiệm với và không có bias
def run_experiment(learning_rates, regularizations, n_epochs_values, with_bias=False):
    best_rmse = float('inf')
    best_params = None
    best_user_factors = None
    best_item_factors = None
    best_user_bias = None
    best_item_bias = None
    global_bias = np.mean(ratings_matrix[ratings_matrix > 0]) if with_bias else None

    # Tạo một DataFrame để lưu kết quả
    results_df = []
    
    for lr in learning_rates:
        for reg in regularizations:
            for n_epochs in n_epochs_values:

                # Khởi tạo các yếu tố ẩn và bias
                user_factors = np.random.normal(0, 0.1, (n_users, n_factors))
                item_factors = np.random.normal(0, 0.1, (n_items, n_factors))
                user_bias = np.zeros(n_users) if with_bias else None
                item_bias = np.zeros(n_items) if with_bias else None

                # Chạy SGD
                user_factors, item_factors, user_bias, item_bias = sgd(
                    ratings_matrix, user_factors, item_factors, user_bias, item_bias, global_bias, lr, reg, n_epochs, with_bias
                )

                # Dự đoán kết quả
                predictions = global_bias + user_bias[:, np.newaxis] + item_bias[np.newaxis, :] + np.dot(user_factors, item_factors.T) if with_bias else np.dot(user_factors, item_factors.T)

                # Đánh giá mô hình
                error = rmse(ratings_matrix, predictions)
                print(f"RMSE with bias={with_bias}: {error}")
                
                # Lưu kết quả vào DataFrame
                results_df = results_df.append({
                    'learning_rate': lr,
                    'regularization': reg,
                    'epochs': n_epochs,
                    'rmse': error
                }, ignore_index=True)
                
                # Lưu lại mô hình với RMSE thấp nhất
                if error < best_rmse:
                    best_rmse = error
                    best_params = (lr, reg, n_epochs)
                    best_user_factors = user_factors
                    best_item_factors = item_factors
                    best_user_bias = user_bias
                    best_item_bias = item_bias
                

    print(f"\nBest model with bias={with_bias} found:")
    print(f"Learning Rate: {best_params[0]}, Regularization: {best_params[1]}, Epochs: {best_params[2]}")
    print(f"Best RMSE with bias={with_bias}: {best_rmse}")

In [73]:
run_experiment(learning_rates, regularizations, n_epochs_values, with_bias=False)

Epoch 1/20 - Loss: 260088.3345048007
Epoch 2/20 - Loss: 260067.9587914076
Epoch 3/20 - Loss: 260047.60058492306
Epoch 4/20 - Loss: 260027.25656172785
Epoch 5/20 - Loss: 260006.9234103933
Epoch 6/20 - Loss: 259986.59782988514
Epoch 7/20 - Loss: 259966.27652776538
Epoch 8/20 - Loss: 259945.9562184041
Epoch 9/20 - Loss: 259925.6336212184
Epoch 10/20 - Loss: 259905.30545888765
Epoch 11/20 - Loss: 259884.9684556026
Epoch 12/20 - Loss: 259864.619335307
Epoch 13/20 - Loss: 259844.25481995192
Epoch 14/20 - Loss: 259823.87162773678
Epoch 15/20 - Loss: 259803.46647136667
Epoch 16/20 - Loss: 259783.03605630837
Epoch 17/20 - Loss: 259762.57707904055
Epoch 18/20 - Loss: 259742.0862253025
Epoch 19/20 - Loss: 259721.56016833533
Epoch 20/20 - Loss: 259700.99556713292
RMSE with bias=False: 4.567956573124355


C:\Users\anhn2\AppData\Local\Temp\ipykernel_28404\318348777.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row_df], ignore_index=True)


Epoch 1/30 - Loss: 260200.01927177902
Epoch 2/30 - Loss: 260179.99661325334
Epoch 3/30 - Loss: 260159.984534492
Epoch 4/30 - Loss: 260139.98015842715
Epoch 5/30 - Loss: 260119.98061920915
Epoch 6/30 - Loss: 260099.98306071103
Epoch 7/30 - Loss: 260079.98463505105
Epoch 8/30 - Loss: 260059.98250114077
Epoch 9/30 - Loss: 260039.97382322518
Epoch 10/30 - Loss: 260019.9557694475
Epoch 11/30 - Loss: 259999.92551041456
Epoch 12/30 - Loss: 259979.88021778382
Epoch 13/30 - Loss: 259959.8170628374
Epoch 14/30 - Loss: 259939.73321508797
Epoch 15/30 - Loss: 259919.62584086138
Epoch 16/30 - Loss: 259899.49210191038
Epoch 17/30 - Loss: 259879.329154015
Epoch 18/30 - Loss: 259859.13414559173
Epoch 19/30 - Loss: 259838.9042162999
Epoch 20/30 - Loss: 259818.63649565898
Epoch 21/30 - Loss: 259798.32810165145
Epoch 22/30 - Loss: 259777.97613932737
Epoch 23/30 - Loss: 259757.57769942182
Epoch 24/30 - Loss: 259737.12985695104
Epoch 25/30 - Loss: 259716.62966980576
Epoch 26/30 - Loss: 259696.07417734858
Ep

KeyboardInterrupt: 

In [ ]:
results_df

In [ ]:
run_experiment(learning_rates, regularizations, n_epochs_values, with_bias=True)

Epoch 1/50 - Loss: 14446.360253983115
Epoch 2/50 - Loss: 14400.696068645804
Epoch 3/50 - Loss: 14358.090970423718
Epoch 4/50 - Loss: 14318.199130403857
Epoch 5/50 - Loss: 14280.723013675324
Epoch 6/50 - Loss: 14245.405669865751
Epoch 7/50 - Loss: 14212.024351424498
Epoch 8/50 - Loss: 14180.385222142848
Epoch 9/50 - Loss: 14150.318961642093
Epoch 10/50 - Loss: 14121.677106866513
Epoch 11/50 - Loss: 14094.329000449434
Epoch 12/50 - Loss: 14068.159239373412
Epoch 13/50 - Loss: 14043.065536584549
Epoch 14/50 - Loss: 14018.956923944403
Epoch 15/50 - Loss: 13995.752237756358
Epoch 16/50 - Loss: 13973.378838611918
Epoch 17/50 - Loss: 13951.771525898206
Epoch 18/50 - Loss: 13930.871614342712
Epoch 19/50 - Loss: 13910.626145729908
Epoch 20/50 - Loss: 13890.987213640048
Epoch 21/50 - Loss: 13871.911382926226
Epoch 22/50 - Loss: 13853.359188816246
Epoch 23/50 - Loss: 13835.294703123396
Epoch 24/50 - Loss: 13817.68515719167
Epoch 25/50 - Loss: 13800.50061294985
Epoch 26/50 - Loss: 13783.7136748987

In [ ]:
results_df

In [42]:
def evaluate_metrics(predictions, ratings_matrix, k=10, threshold=3.5):
    precision_at_k = []
    recall_at_k = []
    f1_at_k = []

    # Duyệt qua từng người dùng (user)
    for user_id in range(n_users):
        # Lấy các sản phẩm đã có đánh giá của user_id
        actual_ratings = ratings_matrix[user_id, :]
        actual_items = np.where(actual_ratings >= threshold)[0]  # Các sản phẩm có rating >= threshold
        
        # Dự đoán xếp hạng cho tất cả các sản phẩm
        predicted_ratings = predictions[user_id, :]
        
        # Lấy K sản phẩm có dự đoán rating cao nhất
        top_k_items = np.argsort(predicted_ratings)[-k:]
        
        # Tính Precision, Recall và F1-Score
        actual_set = set(actual_items)
        predicted_set = set(top_k_items)

        # Tính toán Precision@K, Recall@K và F1-Score@K
        precision = len(actual_set.intersection(predicted_set)) / k
        recall = len(actual_set.intersection(predicted_set)) / len(actual_set) if len(actual_set) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        precision_at_k.append(precision)
        recall_at_k.append(recall)
        f1_at_k.append(f1)

    # Tính trung bình các chỉ số
    avg_precision_at_k = np.mean(precision_at_k)
    avg_recall_at_k = np.mean(recall_at_k)
    avg_f1_at_k = np.mean(f1_at_k)
    return avg_precision_at_k, avg_recall_at_k, avg_f1_at_k


In [ ]:
def evaluate_best_model(best_params, ratings_matrix, n_users, n_items, n_factors=20, k=10, with_bias=False):
    # Lấy các tham số tốt nhất từ kết quả run_experiment
    learning_rate, regularization, n_epochs = best_params

    # Khởi tạo các yếu tố ẩn và bias
    user_factors = np.random.normal(0, 0.1, (n_users, n_factors))
    item_factors = np.random.normal(0, 0.1, (n_items, n_factors))
    user_bias = np.zeros(n_users) if with_bias else None
    item_bias = np.zeros(n_items) if with_bias else None
    global_bias = np.mean(ratings_matrix[ratings_matrix > 0]) if with_bias else None

    # Chạy SGD với bộ tham số tốt nhất
    user_factors, item_factors, user_bias, item_bias = sgd(
        ratings_matrix, user_factors, item_factors, user_bias, item_bias, global_bias, learning_rate, regularization, n_epochs, with_bias
    )

    # Dự đoán kết quả
    predictions = global_bias + user_bias[:, np.newaxis] + item_bias[np.newaxis, :] + np.dot(user_factors, item_factors.T) if with_bias else np.dot(user_factors, item_factors.T)

    # Tính toán Precision@K, Recall@K và F1-Score@K
    precision_at_k, recall_at_k, f1_at_k = evaluate_metrics(predictions, ratings_matrix, k=k)

    # In kết quả
    print(f"Best Model Results:")
    print(f"Precision@{k}: {precision_at_k:.4f}")
    print(f"Recall@{k}: {recall_at_k:.4f}")
    print(f"F1-Score@{k}: {f1_at_k:.4f}")

# Sau khi có bộ tham số tốt nhất từ run_experiment, bạn có thể gọi hàm này
best_params = (0.01, 0.005, 50)  # Ví dụ bộ tham số tốt nhất từ kết quả run_experiment
evaluate_best_model(best_params, ratings_matrix, n_users, n_items, n_factors=20, k=10, with_bias=True)


Epoch 1/50 - Loss: 13755.240960325275
Epoch 2/50 - Loss: 12822.137035648131
Epoch 3/50 - Loss: 12195.022219030001
Epoch 4/50 - Loss: 11672.843986416336
Epoch 5/50 - Loss: 11193.750696229179
Epoch 6/50 - Loss: 10724.240148480327
Epoch 7/50 - Loss: 10241.773233857977
Epoch 8/50 - Loss: 9734.787406869733
Epoch 9/50 - Loss: 9209.193738303922
Epoch 10/50 - Loss: 8688.361167542578
Epoch 11/50 - Loss: 8195.928625097627
Epoch 12/50 - Loss: 7736.6756789786505
Epoch 13/50 - Loss: 7299.324940612033
Epoch 14/50 - Loss: 6872.293418052028
Epoch 15/50 - Loss: 6451.6755220447585
Epoch 16/50 - Loss: 6039.892307341626
Epoch 17/50 - Loss: 5641.791226239047
Epoch 18/50 - Loss: 5261.706936838296
Epoch 19/50 - Loss: 4902.178315244673
Epoch 20/50 - Loss: 4563.967190205253
Epoch 21/50 - Loss: 4246.672170424002
Epoch 22/50 - Loss: 3949.3549595766376
Epoch 23/50 - Loss: 3670.937702547439
Epoch 24/50 - Loss: 3410.3815755824676
Epoch 25/50 - Loss: 3166.737139624466
Epoch 26/50 - Loss: 2939.1405507620398
Epoch 27/